<a href="https://colab.research.google.com/github/takuminmin-m/hisseki_test_py/blob/main/hisseki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ランタイムの接続が切れないようにするJavaScriptコード
DevToolsのコンソールに入力

JavaScript code to stop for the runtime to disconnect 

```javascript
function KeepClicking(){
console.log("Clicking");
document.querySelector("colab-connect-button").click();
}
setInterval(KeepClicking,60000);
```



# データのインポート import data
google driveをマウント

Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ライブラリの読み込みなど

Load libraries and more

In [ ]:
# tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, losses

!pip install -U keras-tuner
import kerastuner as kt

import os
import numpy as np
import IPython.display as display
import random
import matplotlib.pyplot as plt
import pathlib
import IPython

AUTOTUNE = tf.data.experimental.AUTOTUNE

作業ディレクトリの移動とデータの読み込み

Change the working directory and load labels.txt and handwriting images

In [ ]:
%cd /content/drive/MyDrive/colab/hisseki_test_py

def preprocess_image(image):
  image_4ch = tf.image.decode_image(image, channels=4, expand_animations=False)
  image_4ch = tf.image.resize(image_4ch, [128, 128])
  image = tf.cast(tf.reduce_sum(image_4ch, 2, keepdims=True), tf.float32)
  image /= 255.0  # normalize to [0,1] range
  # image_4ch /= 255.0

  return image

def load_and_preprocess_image(path):
  image = tf.io.read_file(path)
  return preprocess_image(image)

label_file = open("tf_datas/labels.txt")
labels_str = label_file.readlines()
label_file.close()
all_labels = list(map(lambda str: int(str), labels_str))

data_num = len(all_labels)
member_num = len(set(all_labels))
image_paths = []
for i in range(data_num):
  image_paths.append("/content/drive/MyDrive/colab/hisseki_test_py/tf_datas/hisseki%d.png" % i)

example_image_path = random.choice(image_paths) # 予めテスト画像を抽出

image_paths = tf.data.Dataset.from_tensor_slices(image_paths)
all_images = list(image_paths.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE))
all_datas = []
for i in range(data_num):
  all_datas.append({"image": all_images[i], "label": all_labels[i]})

# ランダムに画像をテスト表示
# randomly choose a handwriting image and show to test

example_image_raw = tf.io.read_file(example_image_path)
example_image = tf.image.decode_image(example_image_raw, channels=4, expand_animations=False)
plt.imshow(example_image)
plt.grid(False)
plt.show()

教師データの前処理

Preprocess training data

In [ ]:
def split(list, slice_index):
  shuffled_list = list
  random.shuffle(shuffled_list)
  return shuffled_list[:slice_index], shuffled_list[slice_index:]

def separate_image_and_label(list):
  images = []
  labels = []
  for i in range(len(list)):
    images.append(list[i]["image"])
    labels.append(list[i]["label"])
  return images, labels

random.shuffle(all_datas)
slice_index = int(data_num * 0.9)
train_datas, test_datas = split(all_datas, slice_index)

train_images, train_labels = separate_image_and_label(train_datas)
test_images, test_labels = separate_image_and_label(test_datas)

train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

# 識別用モデルを作成 Make a classification model

モデルの構築例

Example model

In [ ]:
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation="relu", input_shape=(128, 128, 4)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.Flatten(),
    layers.Dense(64, activation="relu"),
    layers.Dense(member_num, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

## ハイパーパラメータのチューニング Tune hyperparameter


モデルを作る関数

Function that makes model

VGG16

In [ ]:
def model_builder(hp):
  model = keras.Sequential()

  for i in range(0, 13):
    filter_num = hp.Int(f"filter_num_{i}", 16, 256, 16)
    filter_size = hp.Int(f"filter_size_{i}", 3, 11, 2)

    if i == 0:
      model.add(layers.Conv2D(
          filter_num,
          (filter_size, filter_size),
          padding="same",
          activation="relu",
          input_shape=(128, 128, 4)
      ))
    else:
      model.add(layers.Conv2D(
          filter_num,
          (filter_size, filter_size),
          padding="same",
          activation="relu"
      ))
    
    if i in [1, 3, 6, 9, 12]:
      model.add(layers.MaxPooling2D((2, 2)))

  model.add(layers.Flatten())
  
  for i in range(hp.Int("num_layers", 1, 5, 1)):
    num_units = hp.Int(f"num_units_{i}", 32, 512, 32)
    model.add(layers.Dense(
        units=num_units,
        activation="relu"
    ))

  model.add(layers.Dense(member_num, activation="softmax"))

  hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
  model.compile(
    optimizer=optimizers.Adam(learning_rate=hp_learning_rate),
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
  )

  return model

tuner = kt.Hyperband(
    model_builder,
    objective = "val_accuracy",
    max_epochs = 10,
    directory = "my_dir",
    project_name = "hisseki"
)

コールバックを設定

Set the callbacks

In [ ]:
class ClearTrainingOutput(keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    # self.model.summary()
    # keras.utils.plot_model(self.model, "model_plot.png", show_shapes=True)
    IPython.display.clear_output(wait=True)

探索

Search

In [ ]:
tuner.search(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels), callbacks=[ClearTrainingOutput()])

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

モデルをつくって訓練

Make a model and train it

In [ ]:
model = tuner.hypermodel.build(best_hps)
model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))

モデルをセーブ

Save a model

In [ ]:
import datetime

dt_now = datetime.datetime.now()
timestomp_str = dt_now.strftime("%y%m%d%H%M%S")

model.save(f"classification_hisseki_model{timestomp_str}")

モデルをロード

Load a model

In [ ]:
print("Enter the file name of model")
file_name = input().strip()

model = models.load_model(file_name)

モデルの確認

Check the model

In [ ]:
model.summary()

keras.utils.plot_model(model, "classification_model_plot.png", show_shapes=True)

評価

Evaluation

In [ ]:
model.evaluate(test_images, test_labels, verbose=2)